In [ ]:
from utils import (
    wn, 
    get_normal_form, 
    my_split, 
    TitleInWn,
    is_lat
)
import json
from os import listdir
from os.path import isfile, join
from const import DAMP_OF_WIKIDATA_PATh, GOLD_WIKIDATA_DATASET, ALL_NEED_ARTICLE
from collections import defaultdict
import pickle
import pandas as pd
from tqdm import tqdm
from collections import deque
from xml.dom import minidom
from dataclasses import dataclass

In [ ]:
set_senses = set([' '.join([get_normal_form(w).lower() for w in s.lemma.split()]) for s in wn.senses])

In [ ]:
onlyfiles = [f for f in listdir(DAMP_OF_WIKIDATA_PATh) if isfile(join(DAMP_OF_WIKIDATA_PATh, f))]

In [ ]:
count  = 0
onlyfiles = [f for f in listdir(DAMP_OF_WIKIDATA_PATh) if isfile(join(DAMP_OF_WIKIDATA_PATh, f))]
for file in onlyfiles:
    with open(f'{DAMP_OF_WIKIDATA_PATh}\\{file}', 'r', encoding='utf-8') as f:
        for _, line in enumerate(f):
            page = json.loads(line)
            if (page['id'] and int(page['id'][1:]) > 10000000) or 'ru' in page['descriptions']:
                print(page)
                count += 1
            if count  >= 10:
                break           
        if count  >= 10:
            break

In [ ]:
# with open("D:\\lbase_data\\all_straight_graph.pickle", 'wb') as f:
#     pickle.dump(straight_idx,f)
# with open("D:\\lbase_data\\all_inverse_graph.pickle", 'wb') as f:
#     pickle.dump(inverse_idx,f)

In [ ]:
# with open("D:\\lbase_data\\all_straight_graph.pickle", 'rb') as f:
#     straight_idx = pickle.load(f)
# with open("D:\\lbase_data\\all_inverse_graph.pickle", 'rb') as f:
#     inverse_idx = pickle.load(f)

In [ ]:
# with open("D:\\lbase_data\\clear_inverse_graph.pickle", 'wb') as f:
#     pickle.dump(clear_inverse_idx,f)
# with open("D:\\lbase_data\\clear_inverse_graph.pickle", 'rb') as f:
#     clear_inverse_idx = pickle.load(f)

In [ ]:
tweets = []
to_add = set()
for file in tqdm(onlyfiles):
    with open(f'{DAMP_OF_WIKIDATA_PATh}\\{file}', 'r', encoding='utf-8') as f:
        for line in f:
            info = json.loads(line)
            if info['id'] == 'Q190':
                    print(info)
            if 'ru' in info['label'] and int(info['id'][1:]) <= 10000000 and (is_lat(info["label"]['ru'])) and 'фильм' not in info["label"]['ru']:
                try:
                    label = info["label"]['ru']
                    sense = wn.get_senses(label)
                    if info['id'] == 'Q190':
                            print(sense)        
                    if sense:
                        tweets.append((info, sense[0].lemma))
                        for elem in info['rels']:
                            to_add.add(elem['rel_id'])
                    else:
                        lemma = TitleInWn(set_senses, my_split(label).split(',')[0])
                        if info['id'] == 'Q190':
                            print(lemma, '!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!') 
                        if lemma:
                            tweets.append((info, lemma))
                            for elem in info['rels']:
                                to_add.add(elem['rel_id'])     
                except:
                    pass          

In [ ]:
len(tweets)

In [ ]:
len(to_add)

In [ ]:
for i in tqdm(range(5)):
    if i != 0:
        to_add = new_to_add
    print(len(to_add))
    new_to_add = set()
    for file in onlyfiles:
        with open(f'{DAMP_OF_WIKIDATA_PATh}\\{file}', 'r', encoding='utf-8') as f:
            for line in f:
                info = json.loads(line)
                if info['id'] in to_add and int(info['id'][1:]) <= 10000000 and 'ru' in info['label']  and 'фильм' not in info["label"]['ru']:
                    try:
                        label = info["label"]['ru']
                        sense = wn.get_senses(label)
                        if sense:
                            tweets.append((info, sense[0].lemma))
                        else:
                            lemma = TitleInWn(set_senses, my_split(label).split(',')[0])
                            if lemma is not None:
                                tweets.append((info, lemma))
                        for elem in info['rels']:
                            new_to_add.add(elem['rel_id'])
                    except:
                        pass
                elif info['label'] and info['id'] in to_add:
                    for elem in info['rels']:
                        new_to_add.add(elem['rel_id'])
                    tweets.append((info, info['label']['en']))

In [ ]:
len(tweets)

In [ ]:
# with open(ALL_NEED_ARTICLE, 'wb') as f:
#     pickle.dump(tweets, f)

In [ ]:
data = pd.read_csv(GOLD_WIKIDATA_DATASET)
data

In [ ]:
with open(ALL_NEED_ARTICLE, 'rb') as f:
    articles = pickle.load(f)
len(articles)

In [ ]:
graph_path_straight = defaultdict(set)
graph_path_inverse = defaultdict(set)
id_artircle2idx_article = {}
idx_article2id_artircle = {}
for idx, (article, _) in tqdm(enumerate(articles)):
    id_artircle2idx_article[article['id']] = idx
    idx_article2id_artircle[idx] = article['id']
    for link in article['rels']:
        graph_path_straight[link['rel_id']].add(article['id'])
        graph_path_inverse[article['id']].add(link['rel_id'])

In [ ]:
def find_reachable_vertices(graph, start_vertex):
    visited = set()
    queue = deque([start_vertex])
    reachable_vertices = set()

    while queue:
        current_vertex = queue.popleft()
        visited.add(current_vertex)
        reachable_vertices.add(current_vertex)

        for neighbor in graph[current_vertex]:
            if neighbor not in visited:
                queue.append(neighbor)

    return reachable_vertices

In [ ]:
print(find_reachable_vertices(graph_path_inverse, 'Q4568'))

In [ ]:
def extractCtxS(wn, lemma:str):
    #составляем контекст для слова из wordnet
    ctx_s = set()
    #synonymy
    for sense in wn.get_synsets(lemma):
        for synonymy in sense.senses:
            ctx_s.update(my_split(synonymy.lemma).split(","))
    #Hypernymy/Hyponymy
    for sense in wn.get_senses(lemma):
        for hypernyms in sense.synset.hypernyms:
            ctx_s.update(my_split(hypernyms.title).split(","))
    for sense in wn.get_senses(lemma):
        for hyponyms in sense.synset.hyponyms:
            ctx_s.update(my_split(hyponyms.title).split(","))
    #Sisterhood:
    for sense in wn.get_senses(lemma):
        for hypernyms in sense.synset.hypernyms:
            for sister in hypernyms.hyponyms:
                ctx_s.update(my_split(sister.title).split(","))
    return ctx_s
@dataclass
class WnCtx:
    id: int
    ctx: set
    lemmaInWn: str
    name: str

In [ ]:
file = open("D:\\asd\\ctxS.txt", "rb")
unpickler = pickle.Unpickler(file)
dictWn = unpickler.load()
file.close()

In [ ]:
wn.get_synsets(wn['130555-N-741490'].lemma), wn['130555-N-741490']

In [ ]:
from typing import Tuple, List

In [ ]:
def get_new_lemma(all_synsets, title):
    title = title.lower()
    title = title.replace("—", "-")
    title = title.replace(",", "")
    if title in all_synsets:
        return title
    if "(" in title:
        text = my_split(title).split(",")
        if text[0] in all_synsets:
            return text[0]
    text = title.split(",")
    lemmatized = " ".join([get_normal_form(word).lower()
                for word in text])
    if lemmatized in all_synsets:
        return lemmatized
    if "ё" in title:
        return get_new_lemma(all_synsets, title.replace("ё","е"))
    return None

In [ ]:
all_synsets = set([lem.title.lower() for lem in wn.synsets])

In [ ]:

def create_dict_candidates(articles:List[Tuple[dict, str]]) -> dict:
    """
        Создание словаря из синсета в кандидаты для связывания
    """
    canidates = defaultdict(list)
    for (article, lemma) in tqdm(articles):
        if 'ru' in article['label']:
            if lemma:
                synset_base_on_lemma = wn.get_synsets(lemma)
                if synset_base_on_lemma:
                    for synset in wn.get_synsets(lemma):
                        canidates[synset.id].append((article, lemma))
            else:
                lemma_new = get_new_lemma(all_synsets, article['label']['ru'])
                if lemma_new:
                    for synset in wn.get_synsets(lemma_new):
                        canidates[synset.id].append((article, lemma_new))
    return canidates
candidates = create_dict_candidates(articles)
print(len(candidates))

In [ ]:
# candidates['ЭНЦИКЛОПЕДИЧЕСКИЙ СЛОВАРЬ']
import re


def clean_text(text):
    cleaned_text = re.sub(r"[^\w\s]", "", text)
    cleaned_text = re.sub(r"\d+", "", cleaned_text)
    cleaned_text = re.sub(r"\s+", " ", cleaned_text)
    cleaned_text = cleaned_text.strip().lower()
    return cleaned_text

def extract_ctx_wikidata(article:dict) -> set:
    '''
        Излвечения контекста статьи из ее ооописания на русском
    '''
    ctx = set()
    if 'ru' in article['descriptions']:
        for token in clean_text(article['descriptions']['ru']).split():
            ctx.add(get_normal_form(token).lower())
    return ctx
def get_score(elem1:set, elem2:set) -> float:
    '''
        Функция подсчета близости статьи и контекстов на основе эвристики
    '''
    return (len((elem1 & elem2)) + 1) / (len(elem1) + len(elem2) + 1)

In [ ]:
score_dict = defaultdict(list)
for i, (title_synset, candidatess) in tqdm(enumerate(candidates.items())):
    for candidate, lemma in candidatess:
        if 'N' in wn[lemma][0].id:
            synset_ctx = dictWn[wn[lemma][0].id].ctx
            article_ctx = extract_ctx_wikidata(candidate)
            article_ctx.update([candidate['label']['ru'].lower(), lemma.lower()])
            score = get_score(article_ctx, synset_ctx)
            score_dict[title_synset].append(score)
        else:
            score_dict[title_synset].append(0.0)

In [ ]:
for candidate, lemma in candidates['103410-N']:
        if 'N' in wn[lemma][0].id:
            synset_ctx = dictWn[wn[lemma][0].id].ctx
            article_ctx = extract_ctx_wikidata(candidate)
            article_ctx.update([candidate['label']['ru'], lemma])
            score = get_score(article_ctx, synset_ctx)
            print(candidate['id'],article_ctx)
            score_dict[title_synset].append(score)
        else:
            score_dict[title_synset].append(0.0)

In [ ]:
@dataclass
class display_synset_to_wikidata:
    id:int
    label:int
    lemma:str
    sense_id:int
    score:float
    synset_lemma:str

In [ ]:
res_score={}
for _, (title_synset, candidatess) in tqdm(enumerate(candidates.items())):
    sorted_lst = sorted(zip(candidatess, score_dict[title_synset]), key=lambda x: x[1], reverse=True)
    if sorted_lst and 'N' in title_synset:
        res_score[title_synset] = display_synset_to_wikidata(sorted_lst[0][0][0]['id'], sorted_lst[0][0][0]['label'], sorted_lst[0][0][1], wn[sorted_lst[0][0][1]][0].id, sorted_lst[0][1], title_synset)

In [ ]:
# with open("res.pickle", 'wb') as f:
#     pickle.dump(res_score, f)

In [ ]:
with open("res.pickle", 'rb') as f:
    res_score = pickle.load(f)

In [ ]:
len(res_score)

In [ ]:

data['predict_id'] = data['synset_id'].apply(lambda x: res_score[x].id if x in res_score else 'не связан')
data


In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(data[data['predict_id'] !='не связан'].predict_id.astype(str), data[data['predict_id'] !='не связан'].WikiDataGoldId.astype(str)), data[data['predict_id'] !='не связан'].shape

In [ ]:
sum([1 if i in candidates else 0 for i in data['synset_id'].values]), wn['наставничество'][0].lemma, wn.get_synsets('НАСТАВНИЧЕСТВО')

In [ ]:
id_artircle2idx_article['Q7209862']

In [ ]:
articles[50899]

In [ ]:
candidates['124807-N']

In [ ]:
from src.mapWikipedia import read_dump, create_wikisynset,create_info_about_sense, wn, unambiguous_bindings, read_pkl
from config.const import PATH_TO_TMP_FILE
map = read_pkl(PATH_TO_TMP_FILE+'fst.pkl')

In [1]:
from src.mapWikipedia import read_dump, create_wikisynset,create_info_about_sense, wn, unambiguous_bindings,create_candidates_index_dict, wn, create_candidates_for_multi_stage
from config.const import DAMP_OF_WIKIPEDIA_PATH
# pages, dictPageRedirect, dictRedirect =read_dump(DAMP_OF_WIKIPEDIA_PATH)
# dictWn = create_info_about_sense()
# wiki = create_wikisynset(pages, dictPageRedirect)
# get_sense_id_by_title('Выставка')
# k, n = unambiguous_bindings(wn, dictWn, wiki, mode='read')

In [2]:
dictLemmNew = create_candidates_index_dict(name='lst_candidates_after_snd_stage.pkl', mode='read')


d = create_candidates_for_multi_stage()
len(d), len(dictLemmNew)

Start reading from file
Successful reading
read False
Start reading from file = E:\data_diplom\candidates_for_multi_stage.pkl
Successful reading


(20327, 21576)

In [5]:
d['101712-N'][0].page

Page(id='7', revid='636679', title='Литва', meaningPage=False, multiPage=True, categories=['Газеты Литвы', 'Архитектура Литвы', 'Всемирное наследие в Литве', 'Литва', 'Государства ', 'Государства '], links=['Германская империя', 'Литовская ССР', 'Союз Советских Социалистических Республик', 'Европа', 'Европейский союз', 'литовский язык', 'Вильнюс', 'Смешанная республика', 'Каунас', 'Клайпеда', 'Шяуляй', 'Паневежис', 'Паланга', 'Друскининкай', 'Неринга', 'Бирштонас', 'Тракай', 'Президенты Литвы', 'Гитанас Науседа', 'Ингрида Шимоните', 'Сейм Литовской Республики', 'литовцы', 'Индекс человеческого развития', 'Список стран по индексу человеческого развития', 'евро', 'Общероссийский классификатор валют', 'литовский лит', 'Восточноевропейское время', 'Летнее время', 'Восточноевропейское летнее время', 'государство', 'Северная Европа', 'Северная Европа', 'Список стран по населению', 'Список стран по территории', 'Балтийское море', 'Латвия', 'Белоруссия', 'Польша', 'Калининградская область', 'Р

In [6]:
wn.get_synset_by_id('101712-N')

Synset(id="101712-N", title="ЛИТВА")

In [ ]:
dicttt, wiki = second_stage_bindings()

In [ ]:
dicttt